In [1]:
from IPython.display import display

import os
import numpy as np
from matplotlib import pyplot as plt
from desilike.samples import plotting, Chain
from getdist import plots
import getdist.plots as gdplt
from getdist import MCSamples
import sys
import logging

# Adjusting the sys.path to include the directory containing the Cosmo Blind Module
sys.path.append('../py')
from blinded_cosmology import get_blind_cosmo

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    filename='processing_log.log', 
                    filemode='w')
logger = logging.getLogger()

base_dir = "/pscratch/sd/u/uendert/mocks_restuls/y1_full_shape/"

test_cases = [
        "test_w0-0.9040043101843285_wa0.025634205416364297_fnl20",
         "test_w0-0.9057030601797708_wa-0.6831142329608426_fnl20",
         "test_w0-0.970439944958287_wa-0.507777992481059_fnl20",
         "test_w0-0.996229742129104_wa0.28930866494014884_fnl20",
         "test_w0-1.0485430984101343_wa0.14015686872763022_fnl20",
         "test_w0-1.106392086529483_wa0.45478607672455995_fnl20",
         "test_w0-1.1616966626392298_wa0.3746115553255438_fnl20",
         "test_w0-1.233469858595847_wa0.7658531629974685_fnl20"
]

# Define redshift bins
z_bins = [(0.4, 0.6), (0.6, 0.8), (0.8, 1.1)]

# Define a list of tracers
tracers = ["LRG"]
# Placeholder for the expected values
expected_values_dict = {}

for tracer in tracers:
    for z_min, z_max in z_bins:
        chains = []
        means = []

        for test in test_cases:
            w0_blind = float(test.split("_w0")[1].split("_wa")[0])
            wa_blind = float(test.split("_wa")[1].split("_fnl")[0])
            
            blind_shifts = get_blind_cosmo(tracer, z_min, z_max, w0_blind=w0_blind, wa_blind=wa_blind)
            logger.info(f"Predicted shifts for {tracer} in z_bin = ({z_min}, {z_max}): {blind_shifts}")
            expected_values_dict[test] = blind_shifts

            chain_dir = os.path.join(base_dir, test)
            pattern = f'chain_{tracer}_{z_min}_{z_max}_corr_velocileptors_shapefit-qisoqap_{{}}.npy'
            filenames = [os.path.join(chain_dir, pattern.format(i)) for i in range(8)]
        
            for f in filenames:
                logger.debug(f"Checking file: {f}")
                if not os.path.exists(f):
                    logger.warning(f"File {f} does not exist.")
            
            if all([os.path.exists(f) for f in filenames]):
                gd_chain = Chain.concatenate([Chain.load(f).remove_burnin(0.5)[::20] for f in filenames])
                gd_chain = gd_chain.to_getdist()
                chains.append(gd_chain)

                mean_values = gd_chain.getMeans()
                logger.info(f"Mean values for {test}: {mean_values}")
                means.append(mean_values)

            else:
                logger.warning(f"Skipped {test} due to missing chain files.")
        
            param_names = gd_chain.getParamNames().list()
            for i, mean in enumerate(means):
                mean_values_dict = dict(zip(param_names, mean))
        
        if not chains:
            logger.error(f"No chains were successfully loaded for z_bin = ({z_min}, {z_max}).")
        else:
            legend_labels = [test.split("test_")[1] for test in test_cases]
            
            triangle_plot = gdplt.get_subplot_plotter(width_inch=12)
            triangle_plot.settings.legend_fontsize = 12
            triangle_plot.settings.lab_fontsize = 14
            triangle_plot.settings.axes_fontsize = 12
            triangle_plot.settings.lw_contour = 2
            triangle_plot.settings.alpha_filled_add = 0.5
            
            colors = ['red', 'blue', 'green', 'cyan', 'magenta', 'yellow', 'orange', 'purple']
            triangle_plot.triangle_plot(chains, params=["qiso", "qap", "df", "dm"], analysis_settings={'ignore_rows': 0.3},
                                        filled=True, legend_labels=legend_labels, colors=colors)
            
            plt.suptitle(f"{tracer}: [{z_min},{z_max}]", fontweight="bold", y=1.05)
            plt.show()
            display(plt.gcf())
            plt.close()


parameters = ["qiso", "qap", "df", "dm"]

# Convert means list of arrays to dictionary
means_dict = {test: dict(zip(parameters, mean_array)) for test, mean_array in zip(test_cases, means)}

# Create the LaTeX table
latex_code = "\\begin{table}[ht]\n"
latex_code += "\\centering\n"
latex_code += "\\caption{Test Results}\n"
latex_code += "\\begin{tabular}{|c|c|c|c|c|}\n"
latex_code += "\\hline\n"
latex_code += "Test Case & Parameter & Expected & Mean & Diff \\\\\n"
latex_code += "\\hline\n"

for test in test_cases:
    latex_code += "\\multicolumn{5}{|c|}{" + test + "} \\\\\n"  # Test case name
    latex_code += "\\hline\n"
    for param in parameters:
        expected_value = expected_values_dict[test].get(param, "N/A")
        mean_value = means_dict.get(test, {}).get(param, "N/A")  # Using nested get to avoid KeyError
        diff_value = "N/A"
        if expected_value != "N/A" and mean_value != "N/A":
            diff_value = str(round(mean_value - expected_value, 5))
        
        latex_code += "& " + param + " & " + str(expected_value) + " & " + str(mean_value) + " & " + diff_value + " \\\\\n"
        
    if test != test_cases[-1]:  # If not the last test case, add an extra line for separation
        latex_code += "\\hline\n"

latex_code += "\\end{tabular}\n"
latex_code += "\\end{table}"

print(latex_code)




ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in
ztp: 0.5
Removed no burn in


<Figure size 1200x1200 with 10 Axes>

ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in
ztp: 0.7
Removed no burn in


<Figure size 1200x1200 with 10 Axes>

ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in
ztp: 0.9500000000000001
Removed no burn in


<Figure size 1200x1200 with 10 Axes>

\begin{table}[ht]
\centering
\caption{Test Results}
\begin{tabular}{|c|c|c|c|c|}
\hline
Test Case & Parameter & Expected & Mean & Diff \\
\hline
\multicolumn{5}{|c|}{test_w0-0.9040043101843285_wa0.025634205416364297_fnl20} \\
\hline
& qiso & 1.0217054175773517 & 1.020968760444324 & -0.00074 \\
& qap & 1.0057373485969496 & 1.008451041009546 & 0.00271 \\
& df & 1.1 & -0.012199298979482824 & -1.1122 \\
& dm & 0.0 & 1.3418335996593442 & 1.34183 \\
\hline
\multicolumn{5}{|c|}{test_w0-0.9057030601797708_wa-0.6831142329608426_fnl20} \\
\hline
& qiso & 0.9907923775935199 & 0.9940008621021948 & 0.00321 \\
& qap & 0.9852750171594491 & 0.9996800309515924 & 0.01441 \\
& df & 0.9166367558791577 & -0.058019735157447384 & -0.97466 \\
& dm & 0.0 & 1.0892626902973408 & 1.08926 \\
\hline
\multicolumn{5}{|c|}{test_w0-0.970439944958287_wa-0.507777992481059_fnl20} \\
\hline
& qiso & 0.9855839493141216 & 0.9805756851801938 & -0.00501 \\
& qap & 0.9881076499557144 & 1.020986259055933 & 0.03288 \\
& df & 0.9 